<a href="https://colab.research.google.com/github/MolecularFoundry/HyperspecAnalysis/blob/main/Hyperspectral_Dimensional_Reduction_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hyperspectral Confocal Microscopy  Dimensional Reduction with PCA

This notebook is designed to analyze `hyperspec_picam_mcl` datasets from the `hip_microscope` confocal optical microscope in the Imaging Facility at the Molecular Foundry. The principles and code in this notebook can also be used to analyze other hyperspectral datasets, with some minor changes to loading of the `spec_map`, `h_array`,`v_arra`y, and `wls` data arrays.



## Initial Setup

In [ ]:
# For Google CoLab, in order to access datasets store on Google Drive,
# we must mount the drives on the filesystem. This will ask for your
# permission to share your Google Drive with this notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# To use interactive widgets in CoLab
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Required imports
import h5py
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')


## Loading your Data File

In [ ]:
project_id = "MFP09238"

In [ ]:
dataset_options = glob.glob(f"/content/drive/Shareddrives/{project_id}/Datasets/*/*.h5")

print("Datasets found at the path above are: ")
for ds in dataset_options:
  print(ds)

In [ ]:
# Edit the location of the dataset you want to analyze (you can copy and paste from the output above)

file_path = ""

In [ ]:
# Load ScopeFoundry HDF5
with h5py.File(file_path, 'r') as f:
    M = f['measurement/hyperspec_picam_mcl']
    # 3D array of spectra (2D spatial dimensions)
    spec_map = np.array(M['spec_map'])[0]
    # positions of data points in um along x (horizontal) axis
    h_array = np.array(M['h_array'])
    # positions of data points in um along y (vertical) axis
    v_array = np.array(M['v_array'])
    # wavelengths of spectral values
    wls = np.array(M['wls'])
    # bounds of image
    imshow_extent = np.array(M['imshow_extent'])

In [ ]:
# Store spectra as a flat list and table
df = pd.DataFrame()

Ny, Nx, Nspec = spec_map.shape
X,Y = np.meshgrid(h_array,v_array)
II,JJ = np.meshgrid(np.arange(Nx), np.arange(Ny))
II.reshape(-1).shape
all_spectra = spec_map.reshape(-1,Nspec)
df['x']  = X.reshape(-1)
df['y']  = Y.reshape(-1)
df['ii'] = II.reshape(-1)
df['jj'] = JJ.reshape(-1)
df['total_intensity'] = spec_map.sum(axis=-1).reshape(-1)

display("all_spectra:",all_spectra)
df.head()

## Plot Image and Spectra

In [ ]:
img = spec_map.sum(axis=-1)
plt.figure()
vmin,vmax = np.percentile(img, [1,99])
plt.imshow(img, extent=imshow_extent, origin='lower',vmin=vmin,vmax=vmax)
plt.colorbar()

In [ ]:
avg_spec = spec_map.mean(axis=(0,1))
plt.figure()
plt.plot(wls, avg_spec)
plt.xlabel("wls")
plt.ylabel("intensity")

# Principal Component Analysis

In [ ]:
# Import Scikit Learn PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
hv.extension('bokeh')
# Peform PCA on data
scaledf = StandardScaler().fit_transform(all_spectra)
pca1 = PCA(random_state = 833)
pca1.fit(scaledf)
pca_var_df = pd.DataFrame({"component":range(0,pca1.n_components_ ), "exp_var_ratio":pca1.explained_variance_ratio_})
# Plot the Explain Variance ratio of the first 20 PCA componenents
pca_var = hv.Scatter(pca_var_df.iloc[0:20, :], kdims = ["component"], vdims = ['exp_var_ratio'])
pca_var.opts(size = 6)
display(pca_var)

In [ ]:
# compute component weights on all spectra
N_components = 10
pca_transform = pca1.transform(scaledf)
pcadf = pd.DataFrame(data = pca_transform[:,:N_components], columns = [f'PC{str(int(x))}' for x in range(0,N_components)])

spectra_df_with_pca = pd.concat([df,pcadf],axis=1, join="inner")
display(spectra_df_with_pca)

In [ ]:
# Display the components
N_components = 3
for i in range(N_components):
  plt.plot(wls, pca1.components_[i,:], label=f"PCA Component {i}")
plt.legend()

In [ ]:
# PCA component correlation plots
plt.figure(figsize=(8,8))
for i in range(4):
  for j in range(4):
      plt.subplot(4,4,4*j+i+1)
      plt.title( f"PC{i} vs PC{j}")
      plt.scatter(spectra_df_with_pca[f'PC{i}'], spectra_df_with_pca[f'PC{j}'], marker='.')
      plt.tick_params(left = False, right = False , labelleft = False ,
                      labelbottom = False, bottom = False)

In [ ]:
# PCA component ratio images
Ny, Nx, Nspec = spec_map.shape

plt.figure(figsize=(8,8))
for i in range(4):
  for j in range(4):
      plt.subplot(4,4,4*j+i+1)
      plt.title( f"PC{i} vs PC{j}")

      PCi_map = np.array(spectra_df_with_pca[f'PC{i}']).reshape(Ny,Nx)
      PCj_map = np.array(spectra_df_with_pca[f'PC{j}']).reshape(Ny,Nx)

      ratio_map = PCj_map/PCi_map
      vmin,vmax = np.percentile(ratio_map, [5,95])
      plt.imshow(ratio_map, origin='lower', vmin=vmin,vmax=vmax)
      plt.tick_params(left = False, right = False , labelleft = False ,
                      labelbottom = False, bottom = False)